# BGP Shenanigans

This notebook goes through a set of exanples with a CSR 1000v running IOS-XE 3.17 and the EFT netconf/yang interfaces. As such, please note that some details of the models used may differ as the openconfig-bgp model is not yet support on IOS-XE, meaning that the *native* model is being used.

## Connecting to a Device

Let's define some variables:

In [1]:
#HOST = '172.26.170.252'
#PORT = 830
#USER = 'cisco'
#PASS = 'C1sdevops'

HOST = '172.16.1.222'
PORT = 2022
USER = 'cisco'
PASS = 'cisco'

Now let's establish a NETCONF session to that box using ncclient:

In [2]:
from ncclient import manager

def my_unknown_host_cb(host, fingerprint):
    return True
    
m = manager.connect(host=HOST, port=PORT, username=USER, password=PASS,
                    allow_agent=False,
                    look_for_keys=False,
                    hostkey_verify=False,
                    unknown_host_cb=my_unknown_host_cb)

## Capabilities

Let's look at the capabilities presented by the thing we've just connected to:

In [3]:
print(m.server_capabilities)

['urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-TC-MIB?module=CISCO-CBP-TC-MIB&revision=2008-06-24', 'urn:ietf:params:xml:ns:yang:smiv2:RSVP-MIB?module=RSVP-MIB&revision=1998-08-25', 'urn:ietf:params:xml:ns:yang:smiv2:ATM-FORUM-TC-MIB?module=ATM-FORUM-TC-MIB', 'urn:ietf:params:xml:ns:yang:ietf-packet-fields?module=ietf-packet-fields&revision=2015-07-14', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-ENTITY-EXT-MIB?module=CISCO-ENTITY-EXT-MIB&revision=2008-11-24', 'urn:cisco:params:xml:ns:yang:cisco-qos-common?module=cisco-qos-common&revision=2015-05-09', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-NETSYNC-MIB?module=CISCO-NETSYNC-MIB&revision=2010-10-15', 'http://tail-f.com/ns/mibs/SNMPv2-MIB/200210160000Z?module=SNMPv2-MIB&revision=2002-10-16', 'urn:cisco:params:xml:ns:yang:cisco-environment?module=cisco-environment&revision=2015-04-09', 'http://tail-f.com/ns/mibs/SNMP-NOTIFICATION-MIB/200210140000Z?module=SNMP-NOTIFICATION-MIB&revision=2002-10-14', 'urn:ietf:params:xml:ns:yang:smiv2:CISCO-CBP-

Ok, that's a bit messy, so let's tidy it up a bit and look, initially, at all the base netconf capabilities:

In [4]:
nc_caps = [c for c in m.server_capabilities if c.startswith('urn:ietf:params:netconf')]
for c in nc_caps:
    print(c)

urn:ietf:params:netconf:capability:notification:1.0
urn:ietf:params:netconf:capability:rollback-on-error:1.0
urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
urn:ietf:params:netconf:capability:interleave:1.0
urn:ietf:params:netconf:capability:validate:1.0
urn:ietf:params:netconf:capability:validate:1.1
urn:ietf:params:netconf:base:1.0
urn:ietf:params:netconf:base:1.1
urn:ietf:params:netconf:capability:writable-running:1.0
urn:ietf:params:netconf:capability:xpath:1.0


And now let's look at the capabilities that are related to model support:

In [5]:
import re

for c in m.server_capabilities:
    model = re.search('module=([^&]*)&', c)
    if model is not None:
        print("{}".format(model.group(1)))

        revision = re.search('revision=([0-9]{4}-[0-9]{2}-[0-9]{2})', c)
        if revision is not None:
            print("  revision = {}".format(revision.group(1)))

        deviations = re.search('deviations=([a-zA-Z0-9\-,]+)($|&)',c)
        if deviations is not None:
            print("  deviations = {}".format(deviations.group(1)))

        features = re.search('features=([a-zA-Z0-9\-,]+)($|&)',c)
        if features is not None:
            print("  features = {}".format(features.group(1)))

CISCO-CBP-TC-MIB
  revision = 2008-06-24
RSVP-MIB
  revision = 1998-08-25
ietf-packet-fields
  revision = 2015-07-14
CISCO-ENTITY-EXT-MIB
  revision = 2008-11-24
cisco-qos-common
  revision = 2015-05-09
CISCO-NETSYNC-MIB
  revision = 2010-10-15
SNMPv2-MIB
  revision = 2002-10-16
cisco-environment
  revision = 2015-04-09
SNMP-NOTIFICATION-MIB
  revision = 2002-10-14
CISCO-CBP-TARGET-TC-MIB
  revision = 2006-03-24
DISMAN-EVENT-MIB
  revision = 2000-10-16
MPLS-LSR-STD-MIB
  revision = 2004-06-03
CISCO-FRAME-RELAY-MIB
  revision = 2000-10-13
CISCO-ATM-PVCTRAP-EXTN-MIB
  revision = 2003-01-20
tailf-webui
  revision = 2013-03-07
CISCO-PRODUCTS-MIB
  revision = 2014-11-06
CISCO-UBE-MIB
  revision = 2010-11-29
ietf-access-control-list
  revision = 2015-03-17
MPLS-LDP-GENERIC-STD-MIB
  revision = 2004-06-03
CISCO-AAA-SESSION-MIB
  revision = 2006-03-21
tailf-common-monitoring
  revision = 2013-06-14
CISCO-RF-MIB
  revision = 2005-09-01
ENTITY-MIB
  revision = 2005-08-10
IP-FORWARD-MIB
  revisio

## Enough!! What About Config?

The ncclient library provides for some simple operations. Let's skip thinking about schemas and stuff like that. Instead let's focus on config and getting end setting it. For that, ncclient provides two methods:

* get_config - takes a target data store and an optional filter
* edit_config - takes a target data store and an XML document with the edit request

### But let's define some utilities first!

In [6]:
from lxml import etree

def pretty_print(nc_retval):
        print(etree.tostring(etree.fromstring(nc_retval.data_xml), pretty_print=True))

### Getting Config

Let's look at some simple requests...

In [47]:
c = m.get_config(source='running')
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <device-model-version>
      <major>2</major>
      <minor>1</minor>
      <bug-fix>0</bug-fix>
    </device-model-version>
    <service>
      <timestamps>
        <debug>
          <datetime>
            <msec/>
          </datetime>
        </debug>
        <log>
          <datetime>
            <msec/>
          </datetime>
        </log>
      </timestamps>
    </service>
    <platform>
      <punt-keepalive>
        <disable-kernel-core>false</disable-kernel-core>
      </punt-keepalive>
    </platform>
    <enable>
      <password>
        <secret>C1sdevops</secret>
      </password>
    </enable>
    <username>
      <name>admin</name>
      <password>
        <encryption>0</encryption>
        <password>C1sdevops</password>
      </password>
    </username>
    <username>
      <name>cisco</name>
      <password>
    

Now let's add in a simple filter (this example only works on IOS-XE 16.3.1 and later):

In [12]:
filter = '''
<netconf-yang/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0"/>



### Retieve Route Data

In [48]:
filter = '''
<native>
  <ip>
    <route/>
  </ip>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <ip>
      <route>
        <ip-route-interface-forwarding-list>
          <prefix>0.0.0.0</prefix>
          <mask>0.0.0.0</mask>
          <fwd-list>
            <fwd>172.26.170.1</fwd>
          </fwd-list>
        </ip-route-interface-forwarding-list>
        <vrf>
          <name>Mgmt-vrf</name>
          <ip-route-interface-forwarding-list>
            <prefix>0.0.0.0</prefix>
            <mask>0.0.0.0</mask>
            <fwd-list>
              <fwd>172.26.170.1</fwd>
            </fwd-list>
          </ip-route-interface-forwarding-list>
        </vrf>
      </route>
    </ip>
  </native>
</data>



### Retrieve All BGP Data

Now let's look at the BGP native model:

In [8]:
filter = '''
<native>
  <router>
    <bgp/>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="urn:ios">
    <router>
      <bgp>
        <as-no>1</as-no>
        <bgp>
          <router-id>192.168.0.7</router-id>
          <log-neighbor-changes/>
        </bgp>
        <maximum-paths>
          <max-path>
            <max>1</max>
          </max-path>
          <external-rtfilter>
            <max>1</max>
          </external-rtfilter>
        </maximum-paths>
        <neighbor>
          <id>2001:DB8:B:0:1::3</id>
          <remote-as>1</remote-as>
          <activate/>
          <description>iBGP peer iosv-1</description>
          <password>
            <enctype>0</enctype>
          </password>
          <update-source>
            <Loopback>0</Loopback>
          </update-source>
        </neighbor>
        <neighbor>
          <id>2001:DB8:B:0:1::4</id>
          <remote-as>1</remote-as>
          <activate/>
          <description>iBGP peer iosv-2</d

### Look At A Specific BGP Neighbor

And can we look at a specific neighbor only? Say the one with id (address) '192.168.0.1'?

In [13]:
filter = '''
<native>
  <router>
    <bgp>
      <as-no>1</as-no>
      <neighbor>
        <id>192.168.0.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="urn:ios">
    <router>
      <bgp>
        <as-no>1</as-no>
        <neighbor>
          <id>192.168.0.1</id>
          <remote-as>1</remote-as>
          <description>iBGP peer iosv-1</description>
          <password>
            <enctype>0</enctype>
          </password>
          <update-source>
            <Loopback>0</Loopback>
          </update-source>
        </neighbor>
      </bgp>
    </router>
  </native>
</data>



### Create New BGP Neighbor

Ok, so, yes we can get a specific neighbor. Now, can we create a new neighbor? Let's create one with an id of '192.168.1.1', with a remote-as of 666.

In [16]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="urn:ios">
    <router>
      <bgp>
        <as-no>1</as-no>
        <neighbor nc:operation="create">
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
          <description nc:operation>a new neighbor</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

Now let's pull back that neighbor:

In [21]:
filter = '''
<native>
  <router>
    <bgp>
      <as-no>1</as-no>
      <neighbor>
        <id>192.168.1.1</id>
      </neighbor>
    </bgp>
  </router>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="urn:ios">
    <router>
      <bgp>
        <as-no>1</as-no>
        <neighbor>
          <id>192.168.1.1</id>
          <remote-as>666</remote-as>
          <description>modified description</description>
          <password>
            <enctype>0</enctype>
          </password>
          <update-source>
            <Loopback>0</Loopback>
          </update-source>
        </neighbor>
      </bgp>
    </router>
  </native>
</data>



### Modify The BGP Neighbor Description

Can modify something in the neighbor we just created? Let's keep it simple and modify the description:

In [19]:
from ncclient.operations import TimeoutExpiredError

edit_data = '''
<config>
  <native xmlns="urn:ios">
    <router>
      <bgp>
        <as-no>1</as-no>
        <neighbor>
          <id>192.168.1.1</id>
          <description nc:operation="merge">modified description</description>
        </neighbor>
      </bgp>
    </router>
  </native>
</config>
'''

try:
    edit_reply = m.edit_config(edit_data, target='running', format='xml')
except TimeoutExpiredError as e:
    print("Operation timeout!")
except Exception as e:
    print("severity={}, tag={}".format(e.severity, e.tag))
    print(e)

## Other Stuff

Get interface data from native model:

In [23]:
filter = '''
<native>
  <interface/>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="urn:ios">
    <interface>
      <GigabitEthernet>
        <name>1</name>
        <negotiation>
          <auto>true</auto>
        </negotiation>
        <description>OOB Management</description>
        <cdp>
          <enable>true</enable>
        </cdp>
        <ip>
          <address>
            <primary>
              <address>172.16.1.158</address>
              <mask>255.255.255.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>2</name>
        <negotiation>
          <auto>true</auto>
        </negotiation>
        <description>to iosv-1</description>
        <cdp>
          <enable>true</enable>
        </cdp>
        <ip>
          <address>
            <primary>
              <address>10.0.0.21</address>
              <mask>255.255.255.252</mask>
            </primary>
     

In [26]:
filter = '''
<native>
  <interface>
    <GigabitEthernet>
      <name/>
      <ip/>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="urn:ios">
    <interface>
      <GigabitEthernet>
        <name>1</name>
        <ip>
          <address>
            <primary>
              <address>172.16.1.158</address>
              <mask>255.255.255.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>2</name>
        <ip>
          <address>
            <primary>
              <address>10.0.0.21</address>
              <mask>255.255.255.252</mask>
            </primary>
          </address>
          <ospf>
            <cost>1</cost>
          </ospf>
        </ip>
      </GigabitEthernet>
      <GigabitEthernet>
        <name>3</name>
        <ip>
          <address>
            <primary>
              <address>10.0.0.29</address>
              <mask>255.255.255.252</mask>
            </primary>
          </address>
          <osp

In [142]:
filter = '''
<native>
  <interface>
    <GigabitEthernet>
      <name>1/0/1</name>
    </GigabitEthernet>
  </interface>
</native>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <native xmlns="http://cisco.com/ns/yang/ned/ios">
    <interface>
      <GigabitEthernet>
        <name>1/0/1</name>
        <switchport-conf>
          <switchport>false</switchport>
        </switchport-conf>
        <description>boom</description>
        <ip>
          <address>
            <primary>
              <address>1.1.1.1</address>
              <mask>255.255.254.0</mask>
            </primary>
          </address>
        </ip>
      </GigabitEthernet>
    </interface>
  </native>
</data>



Get interfaces from IETF model:

In [28]:
filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"/>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <description>OOB Management</description>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
      <enabled>true</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>172.16.1.158</ip>
          <netmask>255.255.255.0</netmask>
        </address>
      </ipv4>
      <ipv6 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip"/>
    </interface>
    <interface>
      <name>GigabitEthernet2</name>
      <description>to iosv-1</description>
      <type xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">ianaift:ethernetCsmacd</type>
      <enabled>true</enabled>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>10.0.0.21</ip>
          <n

In [32]:
filter = '''
<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
  <interface>
    <name/>
    <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
      <address><ip/></address>
    </ipv4>
  </interface>
</interfaces>
'''
c = m.get_config(source='running', filter=('subtree', filter))
pretty_print(c)

<data xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
    <interface>
      <name>GigabitEthernet1</name>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>172.16.1.158</ip>
        </address>
      </ipv4>
    </interface>
    <interface>
      <name>GigabitEthernet2</name>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>10.0.0.21</ip>
        </address>
      </ipv4>
    </interface>
    <interface>
      <name>GigabitEthernet3</name>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>10.0.0.29</ip>
        </address>
      </ipv4>
    </interface>
    <interface>
      <name>GigabitEthernet4</name>
      <ipv4 xmlns="urn:ietf:params:xml:ns:yang:ietf-ip">
        <address>
          <ip>10.0.0.25</ip>
        </address>
      </ipv4>
    </interface>
